In [1]:
%run SharedParams.ipynb
%run MerkleTree.ipynb

In [2]:
class Fri:
    
    def __init__(self, cp_eval, fiat_shamir, proof_data):
        assert len(cp_eval) == g_order        
        self.layers = [MerkleTree(cp_eval)]
        proof_data.append(f"oods_eval_merkle:{self.layers[0].root.hex()}")
        commitments.append(self.layers[0].root)

        while True:
            beta = fiat_shamir.get_random_int(commitments)
            new_layer_eval = self.next_eval(self.layers[-1].data, beta)
            if len(new_layer_eval) > RS_factor:
                self.layers.append(MerkleTree(new_layer_eval))
                proof_data.append(f"layer_root:{self.layers[-1].root.hex()}")
                commitments.append(self.layers[-1].root)
            else:
                self.last_layer_const = new_layer_eval[0]
                assert all(self.last_layer_const == val for val in new_layer_eval)
                proof_data.append(f"last_layer_const:{self.last_layer_const}")
                commitments.append(self.last_layer_const)
                break
    
    @staticmethod
    def next_eval(curr_eval, beta):
        coset_dilution = int(g_order / len(curr_eval))
        new_eval = []
        for i in range(len(curr_eval)/2):
            plus_part = (curr_eval[i] + curr_eval[len(curr_eval)/2 + i]) / 2
            minus_part = (curr_eval[i] - curr_eval[len(curr_eval)/2 + i]) / (2 * w**coset_dilution * g**(coset_dilution * i))
            new_eval.append(plus_part + beta * minus_part)
        return new_eval

    def query_layers(self, index, fiat_shamir, proof_data):
        layer_size = len(self.layers[0].data)
        for layer_index in range(len(self.layers)):
            first_eval_index = index % layer_size
            second_eval_index = (index + int(layer_size / 2)) % layer_size
            first_data, first_path = self.layers[layer_index].get_path(first_eval_index)
            second_data, second_path = self.layers[layer_index].get_path(second_eval_index)
            MerkleTree.auth_path(first_data, first_eval_index, first_path, self.layers[layer_index].root, self.layers[layer_index].tree_height)
            MerkleTree.auth_path(second_data, second_eval_index, second_path, self.layers[layer_index].root, self.layers[layer_index].tree_height)
            
            layer_size = int(layer_size / 2)

            proof_data.append(f"layer_first_eval:{first_data}")
            proof_data.append(f"path:{first_path}")
            proof_data.append(f"layer_second_eval:{second_data}")
            proof_data.append(f"path:{second_path}")

In [2]:
# my_eval = []
# for i in range(g_order):
#    my_eval.append(17 * w ** 4 * g**(4*i))

# fri = Fri(my_eval, fiat_shamir, proof_data)


774576236

240647268